In [ ]:
import pandas as pd

In [ ]:
cols=['data','label','index']

# Kannada data

In [ ]:
kannada_train = pd.read_csv('/content/drive/MyDrive/kannada_offensive_train.csv',sep='\t',names=cols)
kannada_dev= pd.read_csv('/content/drive/MyDrive/kannada_offensive_dev.csv',sep='\t',names=cols)
kannada_test = pd.read_csv('/content/drive/MyDrive/kannada_offensive_test.csv',sep='\t',names=['data'])
kannada_train = kannada_train[['data','label']]
kannada_dev = kannada_dev[['data','label']]
kannada_test = kannada_test[['data']]

In [ ]:
kannada_train.head()

,data,label
0,Tik tok alli jagala madtidralla adra baggenu o...,Not_offensive
1,Anyone from kerala here,not-Kannada
2,Movie rerelease madi plss,Not_offensive
3,Amazon prime alli bittidira....yella manele no...,Not_offensive
4,Guru sure news nanu tik tok dawn lod madeda ya...,Not_offensive


In [ ]:
kannada_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6217 entries, 0 to 6216
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   data    6217 non-null   object
 1   label   6217 non-null   object
dtypes: object(2)
memory usage: 97.3+ KB


In [ ]:
print(len(kannada_train))
kannada_train = kannada_train.drop_duplicates()
print(len(kannada_train))
kannada_train['label'].value_counts()

6217
5936


Not_offensive                           3382
not-Kannada                             1407
Offensive_Targeted_Insult_Individual     486
Offensive_Targeted_Insult_Group          327
Offensive_Untargetede                    212
Offensive_Targeted_Insult_Other          122
Name: label, dtype: int64

In [ ]:
kannada_train['token_length'] = [len(x.split(" ")) for x in kannada_train.data]
print(max(kannada_train.token_length))
print(min(kannada_train.token_length))
print(sum(kannada_train.token_length)/len(kannada_train.token_length))

96
1
8.141677897574125


# Tokenization

In [ ]:
!pip install indic-nlp-library

In [ ]:
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git

Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 133 (delta 0), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (133/133), 149.77 MiB | 41.40 MiB/s, done.
Resolving deltas: 100% (51/51), done.


In [ ]:
INDIC_NLP_RESOURCES=r"/content/indic_nlp_resources"
from indicnlp import common
common.set_resources_path(INDIC_NLP_RESOURCES)

In [ ]:
from indicnlp import loader
loader.load()

Normalization

In [ ]:
mal_lines = []
for i in mal_train['data']:
  mal_lines.append(i)
len(mal_lines)

11695

In [ ]:
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory

# input_text="പലദേശം. പല ഭാഷ ഒരേ ഒരു രാജാവ്  അല്ലാതെ  സ്വന്തം രാജവയത് അല്ല"
# remove_nuktas=False
factory=IndicNormalizerFactory()
normalizer=factory.get_normalizer("ml")

# %%time
nor_mal_lines = []
for i in range(len(mal_lines)):
  nor_mal_line = normalizer.normalize(mal_lines[i])
  nor_mal_lines.append(nor_mal_line)
# new_mal_lines 

In [ ]:
len(nor_mal_lines)

11695

Tokenization word level

In [ ]:
from indicnlp.tokenize import indic_tokenize  
tokenized_mal_lines = []
for i in range(len(mal_lines)):
  tokenized_mal_line = indic_tokenize.trivial_tokenize(nor_mal_lines[i])
  tokenized_mal_lines.append(tokenized_mal_line)
# tokenized_mal_lines

In [ ]:
tokenized_mal_lines[2]

['ആരണ്ട',
 'ആരണ്ട',
 'തലുണ്ടാകാണാ',
 'ആരണ്ട',
 'ഞാൻ',
 'ആണ്ട',
 'ഞാൻ',
 'ആണ്ട',
 'ഞാൻ',
 'Royal',
 'Mech',
 'ആടാ',
 'ആരണ്ട',
 'ആരണ്ട',
 'മീശ',
 'പിരിക്കുന്ന',
 'ആരണ്ട',
 'ഞാൻ',
 'ആണ്ട',
 'ഞാൻ',
 'ആണ്ട',
 'ഞാൻ',
 'royal',
 'Mech',
 'ആടാ']

In [ ]:
mal_lines[2]

'ആരണ്ട ആരണ്ട തലുണ്ടാകാണാ ആരണ്ട ഞാൻ ആണ്ട ഞാൻ ആണ്ട ഞാൻ Royal Mech ആടാ  ആരണ്ട ആരണ്ട മീശ പിരിക്കുന്ന ആരണ്ട ഞാൻ ആണ്ട ഞാൻ ആണ്ട ഞാൻ royal Mech ആടാ'

In [ ]:
from indicnlp.transliterate.unicode_transliterate import ItransTransliterator

transliterated_mal_lines = []
flags=[]
for i in range(len(mal_lines)):
  transliterated_mal_line = ItransTransliterator.from_itrans(mal_lines[i],'ml')
  if(transliterated_mal_line == mal_lines[i]):
    flag=1
  else:
    flag=0
  flags.append(flag)
  transliterated_mal_lines.append(transliterated_mal_line)
transliterated_mal_lines[0]


'പലദേശം\u0d64 പല ഭാഷ ഒരേ ഒരു രാജാവ്  അല്ലാതെ  സ്വന്തം രാജവയത് അല്ല'

In [ ]:
print('native malayalam sentences: ',sum(flags))

native malayalam sentences:  1298


In [ ]:
from indicnlp.transliterate.unicode_transliterate import ItransTransliterator

en_transliterated_mal_lines = []
flags=[]
for i in range(len(mal_lines)):
  en_transliterated_mal_line = ItransTransliterator.to_itrans(mal_lines[i],'ml')
  if(en_transliterated_mal_line == mal_lines[i]):
    flag=1
  else:
    flag=0
  flags.append(flag)
  en_transliterated_mal_lines.append(en_transliterated_mal_line)
en_transliterated_mal_lines[0]



'paladesha.m. pala bhaaSha .ore .oru raajaav  allaat.e  svanta.m raajavayat alla'

In [ ]:
whole_mal_train = ''
for i in range(len(mal_lines)):
  whole_mal_train+=str(mal_lines[i])
len(whole_mal_train)

793979

In [ ]:
from indicnlp.langinfo import *

lang='ml'
vowels = 0
for i in range(len(whole_mal_train)):
  if(is_vowel(whole_mal_train[i],lang)):
    vowels+=1
print('Total characters: ',len(whole_mal_train))
print('Total vowels: ',vowels)

Total characters:  793979
Total vowels:  12759


#ULMFiT Kannada

In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 8.6MB/s 


In [ ]:
#reference: https://github.com/goru001/nlp-for-malyalam/blob/master/classification/Malyalam_Classification_Model.ipynb

In [ ]:
from fastai.text import *
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
import sentencepiece as spm
import re
import pdb

In [ ]:
import fastai, torch
fastai.__version__ , torch.__version__

('1.0.61', '1.7.0+cu101')

In [ ]:
class KannadaTokenizer(BaseTokenizer):
    def __init__(self, lang:str):
        self.lang = lang
        self.sp = spm.SentencePieceProcessor()
        self.sp.Load(str('/content/drive/MyDrive/AggressionNLP/Kannada/Tokenizer/kannada_lm.model'))
        
    def tokenizer(self, t:str) -> List[str]:
        return self.sp.EncodeAsPieces(t)

In [ ]:
sp = spm.SentencePieceProcessor()
sp.Load(str('/content/drive/MyDrive/AggressionNLP/Kannada/Tokenizer/kannada_lm.model'))
itos = [sp.IdToPiece(int(i)) for i in range(25000)]

In [ ]:
# 25,000 is the vocab size that we chose in sentencepiece
kannada_vocab = Vocab(itos)

In [ ]:
tokenizer = Tokenizer(tok_func=KannadaTokenizer, lang='kn')

In [ ]:
tokenizer.special_cases

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep']

In [ ]:
label_cols = ['label']

In [ ]:


df_result_2['data']=df_result_2['query']
df_result_2['label']=df_result_2['predicted_label']

kannada_train_new = pd.concat([kannada_train,df_result_2])
kannada_train_new


,data,label,token_length,query,predicted_label,Not_offensive,Offensive_Targeted_Insult_Group,not-Kannada,Offensive_Untargetede,Offensive_Targeted_Insult_Individual,Offensive_Targeted_Insult_Other
0,Tik tok alli jagala madtidralla adra baggenu o...,Not_offensive,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Anyone from kerala here,not-Kannada,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Movie rerelease madi plss,Not_offensive,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Amazon prime alli bittidira....yella manele no...,Not_offensive,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Guru sure news nanu tik tok dawn lod madeda ya...,Not_offensive,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
773,Startup start maadalu capacity growth maadalu ...,Not_offensive,NaN,Startup start maadalu capacity growth maadalu ...,Not_offensive,0.915989,0.0248174,0.0154094,0.0268336,0.00839248,0.00855798
774,Rashmika idanna nodi thi*a urkimbeku🤣🤣🤣🤣🤣,Not_offensive,NaN,Rashmika idanna nodi thi*a urkimbeku🤣🤣🤣🤣🤣,Not_offensive,0.533111,0.0660914,0.00268117,0.117238,0.223934,0.0569445
775,I love you sir,not-Kannada,NaN,I love you sir,not-Kannada,0.151654,0.000889215,0.83936,0.000788491,0.00710054,0.000207254
776,ನೋಡ್ತಾ ಹೋದ್ರೆ ಸಾವಿರಾರು lyrical video ಗಳು,Not_offensive,NaN,ನೋಡ್ತಾ ಹೋದ್ರೆ ಸಾವಿರಾರು lyrical video ಗಳು,Not_offensive,0.789148,0.0331043,0.151223,0.0206078,0.000419387,0.00549806


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(kannada_train_new['data'],kannada_train_new['label'], test_size = 0.2, random_state = 42, stratify=kannada_train_new['label'])

In [ ]:
X_train

769        Ultimate Guru tik tok ಅವರ ಭವಿಷ್ಯ ನೇ ಹೇಳ್ಬಿಟ್ಟೆ
307                            Benki bindangide rasmikage
668                             Kesari prasad Jois b f by
5183                                 Looks very different
2982      Rashmika fans  ND devarkonda fans urkondirtarea
                              ...                        
4021    @Shreenivas K wrang misteek maadtidya.... kurk...
5993    Yake jana ogtila andre bec of telegram app bec...
4507                        Great work bro....nKeep it up
3351               KEERTHAN KUMAR ಅದು ಒಂದ್ ಪದ ಬಿಟ್ಟು ನೋಡು
6053    @VARAPRASAD K K nin Amman thulle shaata bekaad...
Name: data, Length: 5371, dtype: object

In [ ]:
X_train_df = pd.concat([X_train, y_train], axis=1, keys=['text', 'label'])
X_val_df = pd.concat([X_val, y_val], axis=1, keys=['text', 'label'])
X_test_df = pd.concat([kannada_dev['data'], kannada_dev['label']], axis=1, keys=['text', 'label'])

In [ ]:
# X_train_df['text']

In [ ]:
data_lm = TextLMDataBunch.from_df(path='/content', train_df=X_train_df, valid_df=X_val_df, test_df=X_test_df, tokenizer=tokenizer, vocab=kannada_vocab,text_cols='text')

/usr/local/lib/python3.6/dist-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(a, dtype=dtype, **kwargs)


In [ ]:
data_lm.show_batch()

idx,text
0,fer ent ▁x x b os <unk> ▁r ash mi ka ▁f ans <unk> ▁n d ▁de v ar k on da ▁f ans ▁ ur k ond ir ta re a ▁x x b os ▁ಹೌದು ▁ಸರ್ ▁ಅಶ್ವತ್ಥ ಾ ಮ ▁ಇನ್ನೂ ▁ಬದುಕ ಿದ್ದಾನೆ ▁x x b os ▁ನಮ್ಮ ದೇಶ ▁ದ ▁ಕತೆ ▁ಇಷ್ಟ ೆ ▁ಕಣ ೋ ಪ್ಪ ▁x x b os ▁ <unk> v en ki <unk> <unk> d v
1,▁ಮಾತ್ರ . . . ▁ಹು ಶಾ ರ್ . . ▁ಬಾ ರೋ . . ▁d um ▁ಇ ದ್ರೆ ▁ಮೆ ಜೆ ಸ್ಟಿಕ್ ▁x x re p ▁4 ▁ . ▁ಹ ತ್ರಾ ▁ತು ಲ್ಲ ರೆ ▁ ದೆ ಂಗ್ ತಿ ನಿ ▁ಅಷ್ಟೇ ▁ಮಗ ನೇ ▁ನಿಮ್ಮ ▁ಅ ವ್ವ ▁ಹಾ ದ ರಗಿ ತ್ತಿ ▁x x b os <unk> ▁d ub b <unk> ▁f ull ▁m ov ie ▁in ▁ te lu gu ▁x x b os
2,<unk> ▁s ry ▁b ro ▁x x re p ▁4 ▁ . ▁ಥಿಯೇಟರ್ ▁ಅಲ್ಲಿ ▁ನೋಡ ೋ ಕೆ ▁ಕರೋ ನ ▁ಎ ಫೆ ಕ್ಟ್ ▁ಕೊಡ ್ ತಾ ▁ಇದೆ ▁x x re p ▁4 ▁ . ▁p l <unk> ▁ಡೈರೆಕ್ಟರ್ ▁ಬ್ಯಾಂಕ್ ▁ಅ ಕೌ ಂಟ್ ▁ನಂಬರ್ ▁ಕೊಡ ಿ ▁ನ ನ್ ▁ಟಿಕೆಟ್ ▁ದು ಡ್ ▁ಹಾಕ ್ತಿ ನಿ ▁x x b os <unk> ▁c ur i ous ity ▁ thumb a ▁ <unk> a ast hi
3,▁7 ▁ . <unk> vi ch and ran ▁ av ra ▁a a ▁ ha ad ug ala ▁h ab ba ▁n en apa y it u ▁x x re p ▁5 ▁ . <unk> ▁b l ock b us ter ▁ <unk> as n ▁x x b os ▁ಡಿ ▁ಬಾಸ್ ▁ಅಭಿಮಾನಿ ಗಳ ▁ಕಡೆಯಿಂದ ▁all ▁the ▁be st ▁x x re p ▁4 ▁ . ▁x x b os ▁ತುಂಬಾ
4,b os ▁ma y <unk> ▁g od ▁b less ▁you <unk> ▁s han kar ▁x x b os <unk> ▁n ay an appa ' s ▁ book ▁? ? ▁x x b os ▁ <unk> a n il <unk> ▁a ▁ hu ▁ ni am man ▁k al s k o du ▁ ke y du ▁k ol ve ▁ ba a vi ▁ma di ▁k al s thi ni


In [ ]:
awd_lstm_config = awd_lstm_lm_config.copy()
awd_lstm_config['n_hid'] = 1150
learn = language_model_learner(data_lm, arch=AWD_LSTM, drop_mult=0.3, config=awd_lstm_config, pretrained=False)

In [ ]:
# !unzip '/content/drive/MyDrive/AggressionNLP/TamilPretrainedLanguageModel/models.zip' -d '/content/drive/MyDrive/AggressionNLP/TamilPretrainedLanguageModel'

In [ ]:
# Loading the pretrained language model on kannada wikipedia
learn.load('/content/drive/MyDrive/AggressionNLP/Kannada/Pretrained Language Model /ULMFiT/third_kn_lm', with_opt=True)

LanguageLearner(data=TextLMDataBunch;

Train: LabelList (5371 items)
x: LMTextList
▁x x b os <unk> <unk> l t im ate <unk> ru ▁t ik ▁to k ▁ಅವರ ▁ಭವಿಷ್ಯ ▁ನೇ ▁ಹೇಳ ್ ಬಿಟ್ಟ ೆ,▁x x b os <unk> ▁b en ki ▁b ind ang ide ▁ ras mi k age,▁x x b os <unk> ▁k es ari ▁p ras ad <unk> <unk> o is ▁b ▁f ▁by,▁x x b os <unk> ▁l o ok s ▁very ▁d if fer ent,▁x x b os <unk> ▁r ash mi ka ▁f ans <unk> ▁n d ▁de v ar k on da ▁f ans ▁ ur k ond ir ta re a
y: LMLabelList
,,,,
Path: /content;

Valid: LabelList (1343 items)
x: LMTextList
▁x x b os ▁ಅಪ್ಪಟ ▁ಕನ್ನಡ ▁ಚಿತ್ರ ▁ಸೂಪರ್,▁x x b os ▁ನನಗೆ ▁ಮಹಾಭಾರತ ▁ಅಂದರೆ ▁ತುಂಬಾ ನೆ ▁ಇಷ್ಟ ▁x x re p ▁4 ▁ . ▁n ಮಹಾ ಭಾರತದಲ್ಲಿ ▁ಬರುವ ▁ಪಾತ್ರಗಳು,▁x x b os <unk> ▁t og ari ▁t ip pa ▁ ke li ▁t um ba ▁s ant os h ▁ ay it a ▁m att e ▁ old ▁is ▁g old ▁ ant ar all l ▁a du ▁ ni <unk> a ▁ ay it a,▁x x b os <unk> ▁w hy ▁only ▁t ik ▁to k ▁ಅನ್ನ ೋ ▁ತು ಕಾ ಲಿ ▁ನ ▁ಚೈನಾ ▁ಗೆ ▁ಕಳಿಸ ಿ ▁ಅ ವ್ ಳು ▁ಕೂಡ ▁ಚೈನೀಸ್ ▁ ಬ್ರೀಡ್ ▁ಇ ರ್ ಬೇಕು,▁x x b os <unk> <unk> ▁my ▁go d ▁ su per o ▁ su per
y: LMLabelList
,,,,
Path: /content;

Tes

In [ ]:
learn.freeze()

In [ ]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.503750,4.087892,0.272495,00:09


In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(5, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.971847,3.834475,0.305010,00:12
1,3.759426,3.588091,0.343130,00:11
2,3.559414,3.482368,0.360218,00:11
3,3.397767,3.445689,0.365055,00:11
4,3.290018,3.440943,0.366915,00:11


In [ ]:
learn.save_encoder('kn_fine_tuned_enc')

In [ ]:
data_clas = TextClasDataBunch.from_df(path='/content', train_df=X_train_df, valid_df=X_val_df, test_df=X_test_df, tokenizer=tokenizer, vocab=kannada_vocab,text_cols=['text'], label_cols=['label'], bs=16)

/usr/local/lib/python3.6/dist-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(a, dtype=dtype, **kwargs)


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
data_clas.show_batch()


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


text,target
▁x x b os ▁s . g . ▁ma th apa ti ▁t x ▁f d x ff x t <unk> x x x <unk> s r f ▁a ▁ su ▁ <unk> h ▁it ▁do es n ' t <unk> ▁s g ▁it ▁t s c <unk> d c g c x x d f c ▁ ea ts ▁a ▁c <unk> <unk> ▁ su ite ▁d r ▁,not-Kannada
▁x x b os <unk> ▁d iv ▁m ov ie ▁no di ▁ thumb a ▁k us hi ▁a it hu ▁ y ak and re ▁he n ▁ma k lu ▁ <unk> e v ank e ▁at har ava da ▁m ov ie ▁ thumb a ▁ ad bu th ava g it hu ▁d ia ▁ ond ▁s ala ▁no di di ni ▁b t ▁n ang e,Not_offensive
▁x x b os ▁ <unk> t <unk> ro l l <unk> ▁m o n <unk> st <unk> er <unk> k <unk> an <unk> na <unk> da ▁ <unk> v ish al ▁sh et ty ▁ <unk> v in od k al man i ▁ <unk> i ta ly <unk> ▁d a vi ▁ low d an <unk> ▁l o ve <unk> <unk> ▁c and y ▁ಅಂತ ಾ ▁b l,Offensive_Targeted_Insult_Group
▁x x b os ▁ <unk> u n ity ▁of <unk> ▁n am ▁de sh ad alli ▁po or ▁people ▁he e c ch ide . . y ak and re ▁c ast e ▁system ▁ sa lu va g i ▁ av ar ▁h att ir ▁pro per ty ▁he c ch ide ▁ kar an ▁ ava ru ▁h ind e ▁c ast e ▁system ▁me le ▁,Offensive_Targeted_Insult_Individual
▁x x b os ▁ನಮ್ಮ ▁ದೇಶದಲ್ಲಿ ▁ಸ್ವಲ್ಪ ▁ಜನ ▁ದೇಶದ ▁ಬಗ್ಗೆ ▁ಯೋಚನೆ ▁ಮಾಡ ೋ ದು ▁ಬಿಟ್ಟ ▁ . ▁ಬಕೆಟ್ ▁ಹಿಡಿಯ ೋ ▁ಕೆಲಸ ▁ಚ ನ್ನಾಗಿ ▁ಮಾಡ ್ ತಾರೆ ▁ಅಣ್ಣಾ ▁ಅದಕ್ಕೆ ▁ಹೀಗೆ ಲ್ಲ ▁ಮಾಡ ್ ತಾರೆ ▁ಅವರು ▁x x re p ▁4 ▁ . ▁n d esh ak k ag i ▁ <unk> ee va ▁k ot ta ▁a a ▁ y od har ig in ta ▁e ▁t al ent ▁ ank,Offensive_Untargetede


In [ ]:
del awd_lstm_config['tie_weights']
del awd_lstm_config['out_bias']

In [ ]:
learn = text_classifier_learner(data_clas, arch=AWD_LSTM, drop_mult=0.5, config=awd_lstm_config)

In [ ]:
learn.load_encoder('kn_fine_tuned_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (5371 items)
x: TextList
▁x x b os <unk> <unk> l t im ate <unk> ru ▁t ik ▁to k ▁ಅವರ ▁ಭವಿಷ್ಯ ▁ನೇ ▁ಹೇಳ ್ ಬಿಟ್ಟ ೆ,▁x x b os <unk> ▁b en ki ▁b ind ang ide ▁ ras mi k age,▁x x b os <unk> ▁k es ari ▁p ras ad <unk> <unk> o is ▁b ▁f ▁by,▁x x b os <unk> ▁l o ok s ▁very ▁d if fer ent,▁x x b os <unk> ▁r ash mi ka ▁f ans <unk> ▁n d ▁de v ar k on da ▁f ans ▁ ur k ond ir ta re a
y: CategoryList
Not_offensive,Offensive_Targeted_Insult_Individual,not-Kannada,not-Kannada,Offensive_Targeted_Insult_Group
Path: /content;

Valid: LabelList (1343 items)
x: TextList
▁x x b os ▁ಅಪ್ಪಟ ▁ಕನ್ನಡ ▁ಚಿತ್ರ ▁ಸೂಪರ್,▁x x b os ▁ನನಗೆ ▁ಮಹಾಭಾರತ ▁ಅಂದರೆ ▁ತುಂಬಾ ನೆ ▁ಇಷ್ಟ ▁x x re p ▁4 ▁ . ▁n ಮಹಾ ಭಾರತದಲ್ಲಿ ▁ಬರುವ ▁ಪಾತ್ರಗಳು,▁x x b os <unk> ▁t og ari ▁t ip pa ▁ ke li ▁t um ba ▁s ant os h ▁ ay it a ▁m att e ▁ old ▁is ▁g old ▁ ant ar all l ▁a du ▁ ni <unk> a ▁ ay it a,▁x x b os <unk> ▁w hy ▁only ▁t ik ▁to k ▁ಅನ್ನ ೋ ▁ತು ಕಾ ಲಿ ▁ನ ▁ಚೈನಾ ▁ಗೆ ▁ಕಳಿಸ ಿ ▁ಅ ವ್ ಳು ▁ಕೂಡ ▁ಚೈನೀಸ್ ▁ ಬ್ರೀಡ್ ▁ಇ ರ್ ಬ

In [ ]:
learn.freeze()

In [ ]:
learn.loss_func.func

CrossEntropyLoss()

In [ ]:
mcc = MatthewsCorreff()

In [ ]:
learn.metrics = [mcc, accuracy]

In [ ]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,1.014194,0.857088,0.413971,0.679821,00:08


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,0.879197,0.827571,0.472780,0.701415,00:09


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
learn.save('kn_second-full')

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(5, 1e-3, callbacks=[callbacks.SaveModelCallback(learn, every='improvement', monitor='accuracy', name='kn_final')])

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,0.787874,0.798698,0.510456,0.721519,00:20
1,0.823923,0.807420,0.498890,0.718541,00:21
2,0.698181,0.793908,0.529772,0.734177,00:21
3,0.552280,0.803737,0.533151,0.728965,00:18
4,0.425914,0.814373,0.535807,0.725987,00:20


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Better model found at epoch 0 with accuracy value: 0.7215189933776855.


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ar

Better model found at epoch 2 with accuracy value: 0.7341772317886353.


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ar

In [ ]:
learn.load('kn_final')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (5371 items)
x: TextList
▁x x b os <unk> <unk> l t im ate <unk> ru ▁t ik ▁to k ▁ಅವರ ▁ಭವಿಷ್ಯ ▁ನೇ ▁ಹೇಳ ್ ಬಿಟ್ಟ ೆ,▁x x b os <unk> ▁b en ki ▁b ind ang ide ▁ ras mi k age,▁x x b os <unk> ▁k es ari ▁p ras ad <unk> <unk> o is ▁b ▁f ▁by,▁x x b os <unk> ▁l o ok s ▁very ▁d if fer ent,▁x x b os <unk> ▁r ash mi ka ▁f ans <unk> ▁n d ▁de v ar k on da ▁f ans ▁ ur k ond ir ta re a
y: CategoryList
Not_offensive,Offensive_Targeted_Insult_Individual,not-Kannada,not-Kannada,Offensive_Targeted_Insult_Group
Path: /content;

Valid: LabelList (1343 items)
x: TextList
▁x x b os ▁ಅಪ್ಪಟ ▁ಕನ್ನಡ ▁ಚಿತ್ರ ▁ಸೂಪರ್,▁x x b os ▁ನನಗೆ ▁ಮಹಾಭಾರತ ▁ಅಂದರೆ ▁ತುಂಬಾ ನೆ ▁ಇಷ್ಟ ▁x x re p ▁4 ▁ . ▁n ಮಹಾ ಭಾರತದಲ್ಲಿ ▁ಬರುವ ▁ಪಾತ್ರಗಳು,▁x x b os <unk> ▁t og ari ▁t ip pa ▁ ke li ▁t um ba ▁s ant os h ▁ ay it a ▁m att e ▁ old ▁is ▁g old ▁ ant ar all l ▁a du ▁ ni <unk> a ▁ ay it a,▁x x b os <unk> ▁w hy ▁only ▁t ik ▁to k ▁ಅನ್ನ ೋ ▁ತು ಕಾ ಲಿ ▁ನ ▁ಚೈನಾ ▁ಗೆ ▁ಕಳಿಸ ಿ ▁ಅ ವ್ ಳು ▁ಕೂಡ ▁ಚೈನೀಸ್ ▁ ಬ್ರೀಡ್ ▁ಇ ರ್ ಬ

In [ ]:
from sklearn.metrics import accuracy_score, matthews_corrcoef
df_dict = {'query': list(kannada_dev['data']), 'actual_label': list(kannada_dev['label']), 'predicted_label': ['']*kannada_dev.shape[0]}
all_nodes = list(set(kannada_train['label']))
for node in all_nodes:
    df_dict[node] = ['']*kannada_dev.shape[0]
    
i2c = {}
for key, value in learn.data.c2i.items():
    i2c[value] = key
    
df_result = pd.DataFrame(df_dict)
preds = learn.get_preds(ds_type=DatasetType.Test, ordered=True)
for index, row in df_result.iterrows():
    for node in all_nodes:
        row[node] = preds[0][index][learn.data.c2i[node]].item()
    row['predicted_label'] = i2c[np.argmax(preds[0][index]).data.item()]
df_result.head()

,query,actual_label,predicted_label,Not_offensive,Offensive_Targeted_Insult_Group,not-Kannada,Offensive_Untargetede,Offensive_Targeted_Insult_Individual,Offensive_Targeted_Insult_Other
0,100 days pakka,Not_offensive,Not_offensive,0.852287,0.00753121,0.124471,0.00718047,0.00414496,0.00438587
1,Super sir,not-Kannada,not-Kannada,0.280574,0.00280587,0.711139,0.00289403,0.000856695,0.00173078
2,Hands up,not-Kannada,not-Kannada,0.184626,0.00318407,0.79233,0.0155709,0.00133632,0.00295273
3,Sari nivu video na roast madi adre madvaga pub...,Not_offensive,Not_offensive,0.695987,0.131891,0.00300003,0.0516119,0.0787795,0.0387309
4,Krishana shapa tatteleebeku,Not_offensive,Not_offensive,0.832997,0.0378039,0.0180197,0.0777569,0.00757892,0.0258441


In [ ]:
from sklearn.metrics import accuracy_score, matthews_corrcoef

data_lm_2 = TextClasDataBunch.from_df(path='/content',train_df=X_train_df, valid_df=X_val_df,  test_df=kannada_test, tokenizer=tokenizer, vocab=kannada_vocab,text_cols='text',label_cols=['label'])
learn = text_classifier_learner(data_lm_2, arch=AWD_LSTM, drop_mult=0.5, config=awd_lstm_config)
learn.load_encoder('kn_fine_tuned_enc')
learn.load('/content/models/kn_final')

df_dict = {'query': list(kannada_test['text']), 'predicted_label': ['']*kannada_test.shape[0]}
all_nodes = list(set(kannada_train['label']))
for node in all_nodes:
    df_dict[node] = ['']*kannada_test.shape[0]
    
i2c = {}
for key, value in learn.data.c2i.items():
    i2c[value] = key


df_result_2 = pd.DataFrame(df_dict)
preds = learn.get_preds(ds_type=DatasetType.Test, ordered=True)
for index, row in df_result_2.iterrows():
    for node in all_nodes:
        row[node] = preds[0][index][learn.data.c2i[node]].item()
    row['predicted_label'] = i2c[np.argmax(preds[0][index]).data.item()]
df_result_2.head()



/usr/local/lib/python3.6/dist-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(a, dtype=dtype, **kwargs)


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


,query,predicted_label,Not_offensive,Offensive_Targeted_Insult_Group,not-Kannada,Offensive_Untargetede,Offensive_Targeted_Insult_Individual,Offensive_Targeted_Insult_Other
0,ಜೈ ಮೋರಿ ಜೈ ರೋಗಿ ಇಸ್ಲಾಂ ಸಾಯಿಸಿ ಇದೆ ಇವರ ಗುರಿ,Not_offensive,0.996637,0.00120533,8.02004e-05,0.000545592,0.00133549,0.00019684
1,Anna nim e vedio nodinu mathe chaina apps use ...,Not_offensive,0.764424,0.0825607,0.00363173,0.05075,0.0744912,0.0241424
2,Super anna yenu thappila yela sari agi hellidi...,Not_offensive,0.950983,0.0154205,0.00290209,0.0128787,0.0130254,0.00478979
3,Super sir super super super super,not-Kannada,0.445994,0.00323717,0.548967,0.000799711,0.000251789,0.000749716
4,AGT-KELU (KANNADA RAP 2019),not-Kannada,0.110652,0.0046298,0.870419,0.00735278,0.00489435,0.0020528


In [ ]:
accuracy_score(df_result['actual_label'], df_result['predicted_label'])


0.6988416988416989

In [ ]:
matthews_corrcoef(df_result['actual_label'], df_result['predicted_label'])


0.4853800728585681

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(df_result['actual_label'], df_result['predicted_label']))

                                      precision    recall  f1-score   support

                       Not_offensive       0.71      0.84      0.77       426
     Offensive_Targeted_Insult_Group       0.52      0.24      0.33        45
Offensive_Targeted_Insult_Individual       0.75      0.45      0.57        66
     Offensive_Targeted_Insult_Other       0.15      0.12      0.14        16
               Offensive_Untargetede       0.00      0.00      0.00        33
                         not-Kannada       0.72      0.75      0.74       191

                            accuracy                           0.70       777
                           macro avg       0.48      0.40      0.42       777
                        weighted avg       0.66      0.70      0.67       777



In [ ]:
precision    recall  f1-score   support

                       Not_offensive       0.71      0.84      0.77       426
     Offensive_Targeted_Insult_Group       0.52      0.24      0.33        45
Offensive_Targeted_Insult_Individual       0.75      0.45      0.57        66
     Offensive_Targeted_Insult_Other       0.15      0.12      0.14        16
               Offensive_Untargetede       0.00      0.00      0.00        33
                         not-Kannada       0.72      0.75      0.74       191

                            accuracy                           0.70       777
                           macro avg       0.48      0.40      0.42       777
                        weighted avg       0.66      0.70      0.67       777

In [ ]:
df_result.to_excel('kan_ml.xlsx', index=False,encoding='utf-16')

In [ ]:
df_result_2.to_excel('kan_ml_test_preds.xlsx', index=False,encoding='utf-16')

In [ ]:
!mv '/content/models' '/content/drive/MyDrive/AggressionNLP/Kannada Results'